<a href="https://colab.research.google.com/github/WangKun00293/bart-bert-project/blob/main/BART_generate_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
ag_news = load_dataset("ag_news")
## load dataset
import pandas as pd
df_train=ag_news["train"].to_pandas()
test=ag_news["test"].to_pandas()
from sklearn.model_selection import train_test_split
sample = df_train.sample(n=10000, random_state=1)
train, val = train_test_split(sample, test_size=0.1, random_state=311)
train_0=train[train["label"]==0].sample(n=500, random_state=323)
train_1=train[train["label"]==1].sample(n=800, random_state=323)
train_2=train[train["label"]==2].sample(n=1200, random_state=323)
train_3=train[train["label"]==3].sample(n=1500, random_state=323)
df_train=pd.concat([train_0,train_1,train_2,train_3]).sample(frac=1, random_state=324).reset_index(drop=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
def clean_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'[&\\@#\$%\^*()_+={}\[\]:;"\'<>\|`~]', '', text)

    return text

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:
label_map = {0: "Write a World News", 1: "Write a Sport News", 2: "Write a Business News", 3: "Write a Science/Technology News"}
df_train['prompt'] = df_train['label'].map(label_map)
df_remain, df_BART= train_test_split(df_train, test_size=0.5, random_state=325)
df_test, df_val = train_test_split(test,test_size=0.5,random_state=325)
df_remain=df_remain.reset_index(drop=True)
df_BART=df_BART.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)

In [ ]:
df_test

,text,label
0,Egypt Arrests Alleged Sinai Bombers AP AP - Ei...,0
1,Karzai heading for Afghan victory Supporters o...,0
2,Darfur Peace Talks Struggle for Survival ABUJ...,0
3,Apple offers 2004 financial details Apples lat...,3
4,Closing Arguments In Enron Barge Trial Houston...,2
...,...,...
3795,Time to look overseas for a healthier 401k I ...,2
3796,Dell Exits Low-End China Consumer PC Market H...,3
3797,IBM Buys Two Danish Services Firms IBM said Tu...,3
3798,"Been there, won that ND hopes USC game mirrors...",1


## BART

In [ ]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, tokenizer, df, max_length=512):
        self.tokenizer = tokenizer
        self.inputs = df['prompt']
        self.targets = df['text']
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        # Tokenize input and target text
        input_encoding = self.tokenizer(input_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        target_encoding = self.tokenizer(target_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {'input_ids': input_encoding['input_ids'].flatten(),
                'attention_mask': input_encoding['attention_mask'].flatten(),
                'labels': target_encoding['input_ids'].flatten()}


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [ ]:
from torch.utils.data import DataLoader
train_dataset = TextDataset(tokenizer, df_BART)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataset = TextDataset(tokenizer, df_val)
val_loader = DataLoader(val_dataset, batch_size=8)

In [ ]:
## modified on April 1st
from transformers import AdamW
from tqdm.auto import tqdm
import torch
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 8
model.train()

# Initialize a folder path for saving models
folder_path = "/content/drive/My Drive"

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    total_val_loss = 0

    # Training
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1} Training'):
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f'Epoch {epoch+1} Validation'):
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            loss = outputs.loss

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch {epoch+1}: Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

    # Save
    save_path = os.path.join(folder_path, f'model_41_bart_epoch_{epoch + 1}')
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f'Model saved to {save_path}')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 Training:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1 Validation:   0%|          | 0/250 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch 1: Training Loss: 3.1832, Validation Loss: 0.4714
Model saved to /content/drive/My Drive/model_41_bart_epoch_1


...

Epoch 7: Training Loss: 0.3658, Validation Loss: 0.3733
Model saved to /content/drive/My Drive/model_41_bart_epoch_7


Epoch 8 Training:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 8 Validation:   0%|          | 0/250 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch 8: Training Loss: 0.3534, Validation Loss: 0.3723
Model saved to /content/drive/My Drive/model_41_bart_epoch_8


## load model

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/model_41_bart_epoch_6")
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/model_41_bart_epoch_6")

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
model.to(device)

In [ ]:
'''
def generate_text(prompt, num_texts=5, temperature=1.2, top_k=50, top_p=0.95, min_length=50, max_length=200):
    model.eval()
    generated_texts = []
    for _ in range(num_texts):
        # Encode the prompt
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

        # Generate text using the model with specified hyperparameters
        outputs = model.generate(
            input_ids,
            min_length=min_length,
            max_length=max_length,
            do_sample=True,  # Enable sampling
            num_return_sequences=1,  # Generate one sequence at a time within the loop to ensure diversity
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            no_repeat_ngram_size=2,  # Prevents the model from repeating the same n-grams
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode and add the generated text
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(text)

    # Optional: Post-process to filter out too-similar texts
    # This step can involve checking the similarity between texts and only keeping the most unique ones.

    return generated_texts

prompt = "Write a World News"
generated_texts = generate_text(prompt, num_texts=5, temperature=1.2, top_k=50, top_p=0.9, min_length=50, max_length=200)

for text in generated_texts:
    print(text)
'''

Iraqi Forces To Return to Iraq War Iraqi troops will return to Baghdad on Monday, Iraqi forces said on Sunday, a day after they took over the Iraqi army in a bloody offensive that killed hundreds of troops and wounded thousands of civilians in the country.
US to release US troops in Iraq The United States has announced the release of US military forces from Iraq in response to the invasion of Iraq, the Pentagon said on Friday, citing U.S. and Iraqi officials and the Iraqi Defense Forces.
Russia, Iran seek to Make Nuclear Deal The Kremlin is seeking to make nuclear deal with the United States to prevent the spread of nuclear weapons and other weapons in the region, a Kremlin spokesman said on Friday.Russia said it was seeking a deal that would allow it to build nuclear reactors in Iran and Iran.
Iraqi Forces to Defeat Iran Reuters Reuters - The Iraqi army said on Friday it will take up to six months to defeat the Islamic State in Iraq and Syria, a step that would take it to the brink of

In [ ]:
def generate_text_with_beams(prompt, num_texts=5, temperature=1.0, top_k=50, top_p=0.95, min_length=50, max_length=200, num_beams=5, no_repeat_ngram_size=2):
    model.eval()  # Ensure the model is in evaluation mode
    generated_texts = []
    for _ in range(num_texts):
        # Encode the prompt and ensure it's on the correct device
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)

        # Generate text using the model with specified hyperparameters
        outputs = model.generate(
            input_ids,
            min_length=min_length,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=1,  # Generate one sequence at a time to ensure diversity
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=True,  # Enable sampling even with num_beams for diversity
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode and add the generated text
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts

In [ ]:
all_generated_texts = []
desired_texts = 20
num_texts_per_iteration=10
num_iterations = desired_texts // num_texts_per_iteration

for _ in range(num_iterations):
    generated_texts = generate_text_with_beams(
        prompt="write a World news",
        num_texts=num_texts_per_iteration,
        temperature=1.2,
        top_k=50,
        top_p=0.95,
        min_length=50,
        max_length=200,
        num_beams=10,
        no_repeat_ngram_size=2
    )
    all_generated_texts.extend(generated_texts)


In [ ]:
all_generated_texts

['Iraqi troops in Syria Reuters - The Iraqi army said on Friday it was preparing to deploy troops to Syria in response to the invasion of the country by the Islamic militant group Ansar al-Sharia, which has claimed responsibility for the deaths of hundreds of thousands of civilians.',
 'China says it is ready to make a deal with the United States over its nuclear weapons program, China said on Thursday, adding that it has agreed to pay more than US$1 billion to help resolve the nuclear dispute between the two countries.',
 'Russia 39s Foreign Ministry says it is investigating Russian interference in the U.S. presidential election. Reuters - Russian President Vladimir Putin said on Friday that he was looking into Russian meddling in US elections, but did not rule out the possibility that Russia could influence the outcome of the elections.',
 'U.S. seeks to end Iraq War The United States is seeking to impose sanctions on Iraq in response to the invasion of the Islamic Republic of Iraq, 